## Nettoyage des données des Valeurs Foncières 2023 

Notre fichier étant un fichier texte, nous avons besoin de délimiter les données entres elles. Nous pouvons apercevoir que les données sont séparées par | , ce sera notre delimiter. 

In [24]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

df = pd.read_csv('ValeursFoncieres-2023.txt', delimiter='|', low_memory=False)
threshold = 0.5

df = df[df.columns[df.isnull().mean() < threshold]]
df = df.loc[df.isnull().mean(axis=1) < threshold]

print("DataFrame après nettoyage : ", df.shape)
df.info()
df.head()


DataFrame après nettoyage :  (3773998, 21)
<class 'pandas.core.frame.DataFrame'>
Index: 3773998 entries, 0 to 3799406
Data columns (total 21 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   No disposition             int64  
 1   Date mutation              object 
 2   Nature mutation            object 
 3   Valeur fonciere            object 
 4   No voie                    float64
 5   Type de voie               object 
 6   Code voie                  object 
 7   Voie                       object 
 8   Code postal                float64
 9   Commune                    object 
 10  Code departement           object 
 11  Code commune               int64  
 12  Section                    object 
 13  No plan                    int64  
 14  Nombre de lots             int64  
 15  Code type local            float64
 16  Type local                 object 
 17  Surface reelle bati        float64
 18  Nombre pieces principales  float64
 19  Natu

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,...,Code commune,Section,No plan,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,1,05/01/2023,Vente,"1070000,00",184.0,ALL,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,...,354,BD,334,1,3.0,Dépendance,0.0,0.0,NaN,NaN
1,1,05/01/2023,Vente,"1070000,00",159.0,ALL,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,...,354,BD,336,1,2.0,Appartement,233.0,8.0,NaN,NaN
2,1,05/01/2023,Vente,"1070000,00",159.0,ALL,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,...,354,BD,336,1,3.0,Dépendance,0.0,0.0,NaN,NaN
3,1,03/01/2023,Vente,"152200,00",2914.0,RTE,0107,DE PONCIN,1450.0,SERRIERES-SUR-AIN,...,404,D,2317,0,1.0,Maison,64.0,3.0,S,988.0
4,1,05/01/2023,Vente,"269000,00",427.0,CHE,0040,DE L'AUBEPIN,1800.0,SAINT-JEAN-DE-NIOST,...,361,B,2405,0,1.0,Maison,73.0,3.0,S,835.0


Nous gardons les variables suivantes : 

- Date mutation : pour les analyses temporelles des prix dépendant des saisons ou du temps, elle indique la date de la transaction immobilière.

- Nature mutation :

- Valeur foncière : 

- Code postal/Commune/Code commune :

- Nombre de lots :

- Code type local/Type local : 

- Surface réelle bâtie/Nombre prièces principales/Surface terrain : 

In [25]:
columns_to_delete=['No disposition', 'No voie', 'Type de voie', 'Code voie', 'Code plan', 'Section','No plan', 'Nature culture']

df=df.drop(columns_to_delete, errors='ignore', axis=1)
# Supprimer les lignes où il n'y a pas de 'valeur_foncière'
df = df[df['Valeur fonciere'].notnull()]
df.fillna(0, inplace=True)

#df.head()
df.info()
#print(df.isnull().sum())
#print(df.shape)  


<class 'pandas.core.frame.DataFrame'>
Index: 3740127 entries, 0 to 3799406
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Date mutation              object 
 1   Nature mutation            object 
 2   Valeur fonciere            object 
 3   Voie                       object 
 4   Code postal                float64
 5   Commune                    object 
 6   Code departement           object 
 7   Code commune               int64  
 8   Nombre de lots             int64  
 9   Code type local            float64
 10  Type local                 object 
 11  Surface reelle bati        float64
 12  Nombre pieces principales  float64
 13  Surface terrain            float64
dtypes: float64(5), int64(2), object(7)
memory usage: 428.0+ MB


Nous allons également modifier les types des variables pour mieux les manipuler lors des analyses :

- Date mutation devra être de type datetime 
- Nature mutation est préférablement un type catégorie pour optimiser la mémoire
- Valeur fonciere devra être de type float car représentant un montant financier
- Voie est un type category 
- Code postal sera un category
- Commune est un type category
- Code departement est de type category 
- Code commune est de type category.

In [30]:
df['Date mutation']=pd.to_datetime(df['Date mutation'], errors='coerce')
df['Valeur fonciere']=df['Valeur fonciere'].replace({'\xa0':'',',':'.'}, regex=True)
df['Valeur fonciere']=pd.to_numeric(df['Valeur fonciere'], errors='coerce')
df.describe()


,Date mutation,Valeur fonciere,Code postal,Code commune,Nombre de lots,Code type local,Surface reelle bati,Nombre pieces principales,Surface terrain
count,1293669,3.740127e+06,3.740127e+06,3.740127e+06,3.740127e+06,3.740127e+06,3.740127e+06,3.740127e+06,3.740127e+06
mean,2023-06-27 01:37:44.013746176,6.004575e+05,4.975646e+04,2.059664e+02,4.356924e-01,1.325934e+00,3.691220e+01,1.004494e+00,2.052201e+03
min,2023-01-01 00:00:00,1.000000e-01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2023-04-05 00:00:00,6.040000e+04,2.752000e+04,7.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2023-07-02 00:00:00,1.600000e+05,4.900000e+04,1.710000e+02,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.870000e+02
75%,2023-10-02 00:00:00,3.006885e+05,7.500500e+04,2.930000e+02,1.000000e+00,3.000000e+00,4.600000e+01,2.000000e+00,1.032000e+03
max,2023-12-12 00:00:00,7.620000e+08,9.749000e+04,9.090000e+02,1.500000e+02,4.000000e+00,1.865970e+05,9.500000e+01,4.653206e+06
std,NaN,5.094408e+06,2.753415e+04,1.656515e+02,8.530243e-01,1.325822e+00,3.874519e+02,1.784666e+00,1.076411e+04
